<a href="https://colab.research.google.com/github/Nat1490/4GeeksAcademy/blob/master/Preparaci%C3%B3n_para_una_Reuni%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install crewai duckduckgo-search

!pip -q install 'crewai[tools]' decouple langchain-exa exa_py==1.0.7

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["EXA_API_KEY"] = userdata.get("EXA_API_KEY")

# Herramientas

In [ ]:
from exa_py import Exa
from langchain.agents import tool

class ExaSearchTool:

    @tool
    def search(query: str):
        """Search for a webpage based on the query."""
        return ExaSearchTool._exa().search(f"{query}", use_autoprompt=True, num_results=3)

    @tool
    def find_similar(url: str):
        """Search for webpages similar to a given URL.
        The URL passed in should be a URL returned from `search`.
        """
        return ExaSearchTool._exa().find_similar(url, num_results=3)

    @tool
    def get_contents(ids: str):
        """Get the contents of a webpage.
        The ids must be passed in as a list, a list of ids returned from `search`.
        """
        ids = eval(ids)
        contents = str(ExaSearchTool._exa().get_contents(ids))
        print(contents)
        contents = contents.split("URL:")
        contents = [content[:1000] for content in contents]
        return "\n\n".join(contents)

def tools():
    return [ExaSearchTool.search, ExaSearchTool.find_similar, ExaSearchTool.get_contents]

def _exa():
  return Exa(api_key=os.environ["EXA_API_KEY"])


# Agentes

In [ ]:
from textwrap import dedent
from crewai import Agent

class MeetingPreparationAgents():
    def research_agent(self):
        return Agent(
            role="Research Specialist",
            goal="Conduct thorough research on people and companies involved in the meeting",
            tools=ExaSearchTool.tools(),
            backstory=dedent("""
            As a Research Specialist, your mission is to uncover detailed information
            about the individuals and entities participating in the meeting. Your insights
            will lay the groundwork for strategic meeting preparation."""),
            verbose=True
        )

    def industry_analysis_agent(self):
        return Agent(
            role="Industry Analyst",
            goal="Analyze the current industry trends, challenges, and opportunities.",
            tools=ExaSearchTool.tools(),
            backstory=dedent("""
            As an Industry Analyst, your analysis will identify key trends,
            challenges facing the industry, and potential opportunities that
            could be leveraged during the meeting for strategic advantage."""),
            verbose=True
        )
      def meeting_strategy_agent(self):
        return Agent(
            role="Meeting Strategy Advisor",
            goal="Develop talking points, questions, and strategic angles for the meeting",
            tools=ExaSearchTool.tools(),
            backstory=dedent("""
            As a Strategy Advisor, your expertise will guide the development of
            talking points, insightful questions, and strategic angles
            to ensure the meeting's objectives are achieved."""),
            verbose=True
         )
        def summary_and_briefing_agent(self):
          return Agent(
            role="Briefing Coordinator",
            goal="Compile all gathered information into a concise, informative briefing document",
            tools=ExaSearchTool.tools(),
            backstory=dedent("""
            As the Briefing Coordinator, your role is to consolidate the research,
            analysis, and strategic insights."""),
            verbose=True
         )




# Tareas

In [ ]:
from crewai import Task

class MeetingPreparationTasks():
    def research_task(self, agent, participants, context):
        return Task(
            description=dedent(f"""
            Conduct comprehensive research on each of the individuals and companies
            involved in the upcoming meeting. Gather information on recent
            news, achievements, professional background, and any relevant
            business activities.

            Participants: {participants}
            Meeting Context: {context}"""),
            expected_output=dedent("""
            A detailed report summarizing key findings about each participant
            and company, highlighting information that could be relevant for the meeting."""),
            async_execution=True,
            agent=agent
        )
    def industry_analysis_task(self, agent, participants, context):
      return Task(
            description=dedent(f"""
            Analyze the current industry trends, challenges, and opportunities
            relevant to the meeting's context. Consider market reports, recent
            developments, and expert opinions to provide a comprehensive
            overview of the industry landscape.

            Participants: {participants}
            Meeting Context: {context}"""),
            expected_output=dedent("""
            An insightful analysis that identifies major trends, potential
            challenges, and strategic opportunities."""),
            async_execution=True,
            agent=agent
        )
      def summary_and_briefing_task(self, agent, context, objective):
        return Task(
            description=dedent(f"""
            Compile all the research findings, industry analysis, and strategic
            talking points into a concise, comprehensive briefing document for
            the meeting.
            Ensure the briefing is easy to digest and equips the meeting
            participants with all necessary information and strategies.

            Meeting Context: {context}
            Meeting Objective: {objective}"""),
            expected_output=dedent("""
            A well-structured briefing document that includes sections for
            participant bios, industry overview, talking points, and
            strategic recommendations."""),
            agent=agent
    )

In [ ]:
from crewai import Crew

tasks = MeetingPreparationTasks()
agents = MeetingPreparationAgents()

print("## Welcome to the Meeting Prep Crew")
print("-------------------------------")
participants = input("What are the emails for the participants (other than you) in the meeting?\n")
context = input("What is the context of the meeting?\n")
objective = input("What is your objective for this meeting?\n")